In [57]:
from Bio.Seq import Seq, MutableSeq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import random
import json
import os

In [3]:
def generate_gene(length):
    bases = ('A', 'T', 'C', 'G')
    gene = ''.join([random.choice(bases) for _ in range(length)])
    return Seq(gene)

In [4]:
def point_insertion(gene):
    new_gene = MutableSeq(gene)
    new_base = random.choice(('A', 'T', 'C', 'G'))
    new_pos = random.randint(0, len(gene))
    new_gene.insert(new_pos, new_base)
    return Seq(new_gene)

In [5]:
def point_deletion(gene):
    if len(gene) <= 1:
        return gene
    new_gene = MutableSeq(gene)
    pos = random.randint(0, len(gene)-1)
    new_gene = new_gene[:pos] + new_gene[pos+1:]
    return Seq(new_gene)

In [6]:
def point_mutation(gene):
    new_gene = MutableSeq(gene)
    pos = random.randint(0, len(gene)-1)
    old_base = gene[pos]
    base_choices = ['A', 'T', 'C', 'G']     
    base_choices.remove(old_base)
    new_base = random.choice(base_choices)
    new_gene[pos] = new_base
    return Seq(new_gene)

In [7]:
def partial_deletion(gene):
    if len(gene) <=1 :
        return gene
    new_gene = MutableSeq(gene)
    start = random.randint(0, len(gene)-1)
    end = random.randint(start+1, len(gene))
    new_gene = new_gene[start:end]
    return Seq(new_gene)

In [8]:
def random_insertion(gene):
    new_gene = MutableSeq(gene)
    pos = random.randint(0, len(gene))
    seq_len = random.randint(1, len(gene))
    new_seq = generate_gene(seq_len)
    new_gene = new_gene[:pos] + new_seq + new_gene[pos:]
    return Seq(new_gene)

In [9]:
def partial_duplication(gene):
    new_gene = MutableSeq(gene)
    pos = random.randint(0, len(gene)-1)
    seq_len = random.randint(1, len(gene))
    new_seq = new_gene[pos:pos+seq_len]
    new_gene = new_gene[:pos] + new_seq + new_gene[pos:]
    return Seq(new_gene)

In [10]:
def circular_permutation(gene):
    new_gene = MutableSeq(gene)
    pos = random.randint(1, len(gene)-1)
    new_gene = new_gene[pos:] + new_gene[:pos]
    return Seq(new_gene)

In [11]:
def full_duplication(gene):
    new_gene = MutableSeq(gene)
    new_gene = new_gene + new_gene
    return Seq(new_gene)

In [12]:
def to_stop(gene):
    new_gene = MutableSeq(gene)
    stop_pos = new_gene.find('*')
    if stop_pos != -1:
        new_gene = new_gene[:stop_pos]
    return Seq(new_gene)

In [29]:
all_mutations = [
    point_insertion,
    point_deletion,
    point_mutation,
    partial_deletion,
    random_insertion,
    partial_duplication,
    circular_permutation,
    full_duplication,
]

In [30]:
gene = generate_gene(10)

In [31]:
new_genes = {}
for mutation in all_mutations:
    #new_genes[mutation.__name__] = [mutation(gene) for _ in range(10)]
    new_genes[mutation.__name__] = mutation(gene)

In [32]:
new_genes

{'point_insertion': Seq('CCTGCGTTCCC'),
 'point_deletion': Seq('CCTGCTTCC'),
 'point_mutation': Seq('CCTGCGTTCG'),
 'partial_deletion': Seq('TTCC'),
 'random_insertion': Seq('CCTGCGCTTAATTTGTTCC'),
 'partial_duplication': Seq('CCTGCGTTGCGTTCC'),
 'circular_permutation': Seq('CCCCTGCGTT'),
 'full_duplication': Seq('CCTGCGTTCCCCTGCGTTCC')}

In [33]:
def save_protein(protein, ident, desc, filename):
    record = SeqRecord(protein, id=ident, description=desc)
    SeqIO.write(record, filename, 'fasta')

In [35]:
for mutation, gene in new_genes.items():
    protein = gene.translate(stop_symbol="")
    save_protein(protein, mutation, 'gen 1', f'mutations/{mutation}.fasta')

In [36]:
input_dir = 'data' #@param {type:"string"}
result_dir = 'output' #@param {type:"string"}

# number of models to use
#@markdown ---
#@markdown ### Advanced settings
msa_mode = "single_sequence" #@param ["MMseqs2 (UniRef+Environmental)", "MMseqs2 (UniRef only)","single_sequence","custom"]
num_models = 1 #@param [1,2,3,4,5] {type:"raw"}
num_recycles = 3 #@param [1,3,6,12,24,48] {type:"raw"}
stop_at_score = 100 #@param {type:"string"}
#@markdown - early stop computing models once score > threshold (avg. plddt for "structures" and ptmscore for "complexes")
use_custom_msa = False
num_relax = 0 #@param [0, 1, 5] {type:"raw"}
use_amber = num_relax > 0
relax_max_iterations = 200 #@param [0,200,2000] {type:"raw"}
use_templates = False #@param {type:"boolean"}
do_not_overwrite_results = True #@param {type:"boolean"}
zip_results = False #@param {type:"boolean"}

In [37]:
import sys

from colabfold.batch import get_queries, run
from colabfold.download import default_data_dir
from colabfold.utils import setup_logging
from pathlib import Path

# For some reason we need that to get pdbfixer to import
if use_amber and f"/usr/local/lib/python{python_version}/site-packages/" not in sys.path:
    sys.path.insert(0, f"/usr/local/lib/python{python_version}/site-packages/")

setup_logging(Path(result_dir).joinpath("log.txt"))

In [61]:
def fold(input_dir, result_dir):
    queries, is_complex = get_queries(input_dir)
    results = run(
        queries=queries,
        result_dir=result_dir,
        use_templates=use_templates,
        num_relax=num_relax,
        relax_max_iterations=relax_max_iterations,
        msa_mode=msa_mode,
        model_type="auto",
        num_models=num_models,
        num_recycles=num_recycles,
        model_order=[1, 2, 3, 4, 5],
        is_complex=is_complex,
        data_dir=default_data_dir,
        keep_existing_results=do_not_overwrite_results,
        rank_by="auto",
        pair_mode="unpaired+paired",
        stop_at_score=stop_at_score,
        zip_results=zip_results,
        user_agent="colabfold/google-colab-batch",
    )
    return results

In [62]:
results = fold('mutations', 'test')
print(results)

2024-11-21 16:17:53,363 Running on GPU
2024-11-21 16:17:53,383 Found 2 citations for tools or databases
2024-11-21 16:17:53,384 Query 1/8: partial_deletion (length 1)
2024-11-21 16:17:59,319 Padding length to 6
2024-11-21 16:18:08,249 alphafold2_ptm_model_1_seed_000 recycle=0 pLDDT=50.6 pTM=0.00224
2024-11-21 16:18:08,316 alphafold2_ptm_model_1_seed_000 recycle=1 pLDDT=15.5 pTM=0.000328 tol=0.0001
2024-11-21 16:18:08,384 alphafold2_ptm_model_1_seed_000 recycle=2 pLDDT=15.5 pTM=0.000279 tol=0.0001
2024-11-21 16:18:08,451 alphafold2_ptm_model_1_seed_000 recycle=3 pLDDT=15.5 pTM=0.000291 tol=0.0001
2024-11-21 16:18:08,452 alphafold2_ptm_model_1_seed_000 took 9.1s (3 recycles)
2024-11-21 16:18:08,461 reranking models by 'plddt' metric
2024-11-21 16:18:08,462 rank_001_alphafold2_ptm_model_1_seed_000 pLDDT=15.5 pTM=0.000291
2024-11-21 16:18:08,779 Query 2/8: circular_permutation (length 3)
2024-11-21 16:18:09,991 Padding length to 6
2024-11-21 16:18:10,047 alphafold2_ptm_model_1_seed_000 rec

In [ ]:
fitnesses = []
for metrics in results['metric']:
    mean_plddt = metrics[0]['mean_plddt']
    ptm = metrics[0]['ptm']
    fitness = calculate_fitness(mean_plddt, ptm)
    fitnesses.append(fitness)


In [67]:
def calculate_fitness(mean_plddt, ptm):
    return (mean_plddt / 100) * 0.5 + ptm * 0.5

In [55]:
calculate_fitness('point_deletion')

0.17051666666666665

In [59]:
all_fitnesses = {
    mutation.__name__: calculate_fitness(mutation.__name__)
    for mutation in all_mutations
}

In [60]:
all_fitnesses

{'point_insertion': 0.10254999999999999,
 'point_deletion': 0.17051666666666665,
 'point_mutation': 0.10254999999999999,
 'partial_deletion': 0.07725,
 'random_insertion': 0.36377500000000007,
 'partial_duplication': 0.24752999999999997,
 'circular_permutation': 0.13785000000000003,
 'full_duplication': 0.3197416666666666}

## End goals
* List of mutated genes along with their predicted folds
    * Fasta file with associated pdb file
* Metadata with gene seq : [protein fasta file, folded pdb file, fitness, generation, parent(s)]